# <center> Work calculation of sMAE and test_predict for all stores

In [1]:
import os
workdir = ".."
os.chdir(workdir)

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# plt.style.use('fivethirtyeight')
import json
import tqdm
import warnings

In [3]:
from models.forecast import SalesForecast

In [4]:
path_to_model = "model_settings"
data_path = "data"

train_file = "train.csv"
test_file = "test.csv"

### function

In [5]:
def read_data(store_id, file_name=train_file, path=data_path):
    data = pd.read_csv(os.path.join(path, file_name), index_col=1, parse_dates=True)
    data = data[data.id == store_id].drop("id", axis=1)
    return data

In [6]:
def fill_vacation(data):
    ind = data.index
    full_ind = pd.date_range(ind[0], ind[-1], freq='D')
    data = pd.DataFrame(data, index=full_ind)
    data = data.fillna(method="ffill")
    return data

In [7]:
def data_preprocessing(data, store_id):
    data = fill_vacation(data)
    if store_id in [4, 8, 10, 13, 17]:
        data = drop_days(data)
    return data

In [8]:
def drop_days(data, num_dropped_days=224):
    """
    Deletes days where data has been missing for 7 months. 
    For correct loading into the model, the dates are shifted to be continuous. 
    In this case, the days of the week and the parity of the week number are preserved. 
    This is achieved by the correct number of deleted days 
    (equal to 224) - a multiple of 7 and the number of weeks is even: 224/7 = 32
    """
    data = data.copy()
    dropped_date = pd.date_range("2016-06-01", periods=num_dropped_days, freq="D")
    data = data.drop(dropped_date)
    ind = pd.date_range(end=data.index[-1], periods=len(data))
    data.index = ind
    return data

In [9]:
def data_split(data, num_days=31):
    train, valid = data[:-num_days].copy(), data[-num_days:].copy()
    return train, valid

In [10]:
def calc_sMAE(predict, valid, train):
    sMAE = (predict["forecast"] - valid["target"]).abs().mean() / train["target"].abs().mean()
    return sMAE

## <center>Calculation sMAE for all stores

In [11]:
%%time
warnings.filterwarnings('ignore')

sMAEs = {}
for store_id in tqdm.tqdm(range(20), total=20):
    prediction_horizon = 31
    
    data = read_data(store_id)
    data = data_preprocessing(data, store_id)
    train, valid = data_split(data, prediction_horizon)
    
    hyperparams_file_name = f"hyperparams_{store_id}.json"    
    with open(os.path.join(path_to_model, hyperparams_file_name)) as file:
        params = json.load(file)
    
    sales_forecast = SalesForecast(train, params=params)
    sales_forecast.fit()
    
    predict = sales_forecast.predict(prediction_horizon)
    sMAE = calc_sMAE(predict, valid, train)
    
    sMAEs[store_id] = sMAE
warnings.filterwarnings('default')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [07:50<00:00, 23.52s/it]

Wall time: 7min 50s


In [12]:
sMAEs

{0: 0.13302007386771642,
 1: 0.15570656046221154,
 2: 0.13296007318556652,
 3: 0.18103777497452464,
 4: 0.1750562131822194,
 5: 0.21175680875936226,
 6: 0.1838098040936155,
 7: 0.13950192940321227,
 8: 0.1791491263083369,
 9: 0.17169703536195088,
 10: 0.13484542826029203,
 11: 0.16848449298482127,
 12: 0.22347717122285637,
 13: 0.11284518991307839,
 14: 0.14371403452528145,
 15: 0.153525128273715,
 16: 0.18534914446680595,
 17: 0.18511088264892495,
 18: 0.17106326352930915,
 19: 0.15257524814377113}

## <center>Calculation predicts on full data for all stores and save to test_predict.csv

In [13]:
def save_predict_to_testfile(predict, store_id, test_predict_file="test_predict.csv"):
    test_predict = pd.read_csv(os.path.join(data_path, test_predict_file), index_col=0)
    test_predict.loc[store_id, "target"] = predict.loc[test_predict[test_predict.index == store_id].dt].values
    test_predict.to_csv(os.path.join(data_path, test_predict_file))

In [14]:
%%time
warnings.filterwarnings('ignore')

sMAEs = {}
for store_id in tqdm.tqdm(range(20), total=20):
    data = read_data(store_id)
    data = data_preprocessing(data, store_id)
    
    hyperparams_file_name = f"hyperparams_{store_id}.json"    
    with open(os.path.join(path_to_model, hyperparams_file_name)) as file:
        params = json.load(file)
    
    sales_forecast = SalesForecast(data, params=params)
    sales_forecast.fit()
    
    prediction_horizon = 31
    predict = sales_forecast.predict(prediction_horizon)
    save_predict_to_testfile(predict, store_id, test_predict_file="test_predict.csv")

    
warnings.filterwarnings('default')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [07:46<00:00, 23.33s/it]

Wall time: 7min 46s


## Выводы:

- для прогнозов была использована модель семейства SARIMA
- подбор гиперпараметров для store_id = 0 был проведен в ноутбуке `02_model_building_(id=0).ipynb`, 
включая **подробные комментарии и визуализации**
- для получения итогового результата по каждому магазину был выполненен отдельный подбор гиперпараметров, см. папку `notebooks/parameters_selection`. Параметры по каждому магазину сохранены в папке `model_settings`
- для прогнозов модель была упакована в соответствующий класс
- для прогнозов были сформированы дополнительные фичи
- в настоящем ноутбуке были сделаны прогнозы и валидации с помощью метрики sMAE для всез магазинов
- тестовые прогнозы для всех магазинов посчитаны и сохранены в файл `test_predict.csv` в папке `data`